In [119]:
# Data Cleaning Notebook - Titanic Dataset

## 1. Import Libraries
import pandas as pd
import numpy as np
from IPython.display import display

## 2. Load Dataset
df = pd.read_csv("../data/raw_dataset.csv")
print("Dataset shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

Dataset shape: (891, 12)

First 5 rows:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [120]:
## 3. Initial Exploration

print("=== Dataset Information ===")
df.info()

print("\n=== Summary Statistics ===")
from IPython.display import display
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
display(df.describe(include='all'))

print("\n=== Missing Values ===")
missing_values = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percent
})
display(missing_info[missing_info['Missing Count'] > 0])

print("\n=== Duplicate Rows ===")
print("Duplicate rows:", df.duplicated().sum())

=== Dataset Information ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

=== Summary Statistics ===


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Dooley, Mr. Patrick",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN



=== Missing Values ===


,Missing Count,Missing Percentage
Age,177,19.865320
Cabin,687,77.104377
Embarked,2,0.224467



=== Duplicate Rows ===
Duplicate rows: 0


In [121]:
## 4. Handle Missing Values

print("Before handling missing values:")
print(f"Dataset shape: {df.shape}")
print("Missing values per column:")
print(df.isnull().sum())

# Approach (updated):
# - Age: Fill with median (less sensitive to outliers)
# - Cabin: Fill missing with string 'None'
# - Embarked: Fill with mode (most frequent)
# - Title: Extract from Name; remove honorific tokens from Name but keep Title column

# Handle Age - fill with median
age_median = df['Age'].median()
age_missing_before = df['Age'].isnull().sum()
df['Age'] = df['Age'].fillna(age_median)
age_filled_count = age_missing_before
print(f"Filled {age_filled_count} missing Age values with median: {age_median}")

# Handle Embarked - fill with mode
embarked_mode = df['Embarked'].mode()[0]
embarked_missing_before = df['Embarked'].isnull().sum()
df['Embarked'] = df['Embarked'].fillna(embarked_mode)
embarked_filled_count = embarked_missing_before
print(f"Filled {embarked_filled_count} missing Embarked values with mode: {embarked_mode}")

# Handle Cabin - keep column, fill missing with 'None'
df['Cabin'] = df['Cabin'].fillna('None')
print("Filled missing 'Cabin' with 'None'")

print("\nAfter handling missing values:")
print(f"Dataset shape: {df.shape}")
print("Remaining missing values:")
print(df.isnull().sum())

Before handling missing values:
Dataset shape: (891, 12)
Missing values per column:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Filled 177 missing Age values with median: 28.0
Filled 2 missing Embarked values with mode: S
Filled missing 'Cabin' with 'None'

After handling missing values:
Dataset shape: (891, 12)
Remaining missing values:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [122]:
## 5. Remove Duplicates

print("Before duplicate removal:")
print(f"Dataset shape: {df.shape}")
print(f"Duplicate rows: {df.duplicated().sum()}")

df = df.drop_duplicates()

print("\nAfter duplicate removal:")
print(f"Dataset shape: {df.shape}")
print(f"Remaining duplicate rows: {df.duplicated().sum()}")

Before duplicate removal:
Dataset shape: (891, 12)
Duplicate rows: 0

After duplicate removal:
Dataset shape: (891, 12)
Remaining duplicate rows: 0


In [123]:
## 6. Standardize Formats

print("Before standardization:")
print("Unique values in categorical columns:")
print("Sex:", df['Sex'].unique())
print("Embarked:", df['Embarked'].unique())

# Standardize text formats
df['Sex'] = df['Sex'].str.strip().str.lower()
df['Embarked'] = df['Embarked'].str.strip().str.upper()

print("\nAfter standardization:")
print("Sex:", df['Sex'].unique())
print("Embarked:", df['Embarked'].unique())

# Fix Name column formatting
# Extract Title first, then strip honorific from Name
# Use raw string to avoid invalid escape warnings
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 
                                    'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

# Clean Name: title case and remove the honorific token like ' Mr.' or ' Mrs.'
df['Name'] = df['Name'].str.title()
df['Name'] = df['Name'].str.replace(r'\s(Mr|Mrs|Miss|Master|Rare)\.', '', regex=True)

print("\nTitle distribution:")
print(df['Title'].value_counts())

Before standardization:
Unique values in categorical columns:
Sex: ['male' 'female']
Embarked: ['S' 'C' 'Q']

After standardization:
Sex: ['male' 'female']
Embarked: ['S' 'C' 'Q']

Title distribution:
Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64


In [124]:
# Removed duplicate reload: keep using the existing df from earlier cells
print("(Skipping reload) Continuing with existing dataset. Current shape:", df.shape)
from IPython.display import display
display(df.head())

(Skipping reload) Continuing with existing dataset. Current shape: (891, 13)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,Mr
1,2,1,1,"Cumings, John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,Miss
3,4,1,1,"Futrelle, Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, William Henry",male,35.0,0,0,373450,8.0500,None,S,Mr


In [125]:
## 7. Outlier Detection & Treatment

print("=== Outlier Analysis ===")

# IQR method for Fare outliers
Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Fare - Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
print(f"Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")

outliers_fare = df[(df['Fare'] < lower_bound) | (df['Fare'] > upper_bound)]
print(f"Number of Fare outliers: {len(outliers_fare)}")

# Cap outliers instead of removing to preserve data
df['Fare'] = np.where(df['Fare'] > upper_bound, upper_bound, df['Fare'])
df['Fare'] = np.where(df['Fare'] < lower_bound, lower_bound, df['Fare'])

print("\nAfter outlier treatment:")
print(f"Fare range: {df['Fare'].min():.2f} to {df['Fare'].max():.2f}")

=== Outlier Analysis ===
Fare - Q1: 7.91, Q3: 31.00, IQR: 23.09
Lower bound: -26.72, Upper bound: 65.63
Number of Fare outliers: 116

After outlier treatment:
Fare range: 0.00 to 65.63


In [126]:
## 8. Final Data Quality Check

print("=== Final Data Quality Check ===")
print(f"Final dataset shape: {df.shape}")
print("\nData types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())
print("\nDuplicate rows:", df.duplicated().sum())

print("\n=== Before vs After Comparison ===")
print("Initial shape:", pd.read_csv("../data/raw_dataset.csv").shape)
print("Final shape:", df.shape)

print("\nSample of cleaned data:")
from IPython.display import display
display(df.head())

=== Final Data Quality Check ===
Final dataset shape: (891, 13)

Data types:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Title           object
dtype: object

Missing values:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Title          0
dtype: int64

Duplicate rows: 0

=== Before vs After Comparison ===
Initial shape: (891, 12)
Final shape: (891, 13)

Sample of cleaned data:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,Mr
1,2,1,1,"Cumings, John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,65.6344,C85,C,Mrs
2,3,1,3,"Heikkinen, Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,Miss
3,4,1,1,"Futrelle, Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, William Henry",male,35.0,0,0,373450,8.0500,None,S,Mr


In [127]:
## 9. Save Cleaned Dataset
# Ensure Has_Cabin is not present before saving
if 'Has_Cabin' in df.columns:
    df = df.drop('Has_Cabin', axis=1)

df.to_csv("../data/cleaned_dataset.csv", index=False)
print("Cleaned dataset saved to '../data/cleaned_dataset.csv'")

# Save summary statistics
print("\n=== Summary Statistics of Cleaned Data ===")
from IPython.display import display
display(df.describe())

Cleaned dataset saved to '../data/cleaned_dataset.csv'

=== Summary Statistics of Cleaned Data ===


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,24.046813
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,20.481625
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,65.634400
